Predict Handwritten Digit Using GUI

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("digit_cnn_model.h5")

# Preprocess image for model
def preprocess_image(path):
    img = Image.open(path).convert("L")  # grayscale
    img = img.resize((28, 28))
    img = np.array(img)
    img = 255 - img  # Invert if needed
    img = img / 255.0
    img = img.reshape(1, 28, 28, 1)
    return img

# Predict digit
def predict_digit():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return

    # Display image
    img = Image.open(file_path)
    img_resized = img.resize((150, 150))
    img_display = ImageTk.PhotoImage(img_resized)
    canvas.create_image(75, 75, image=img_display)
    canvas.image = img_display

    # Preprocess and predict
    preprocessed = preprocess_image(file_path)
    prediction = model.predict(preprocessed)
    digit = np.argmax(prediction)

    result_label.config(text=f"Predicted Digit: {digit}")

# Tkinter UI
root = tk.Tk()
root.title("Handwritten Digit Recognizer")
root.geometry("300x350")

canvas = tk.Canvas(root, width=300, height=300)
canvas.pack(pady=10)

upload_btn = tk.Button(root, text="Upload Digit Image", command=predict_digit)
upload_btn.pack(pady=10)

result_label = tk.Label(root, text="Predicted Digit: ", font=("Arial", 16))
result_label.pack(pady=10)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


Predict Handwritten Digit by Giving Image Path

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model
model = load_model("digit_cnn_model.h5")

# Load input image
img = cv2.imread(r"C:\Users\porje\Downloads\eq.jpg", cv2.IMREAD_GRAYSCALE)  # Change to your file name

if img is None:
    print("❌ Image not found.")
    exit()

# Resize and threshold
img = cv2.resize(img, (300, 100))  # Wider size to fit multiple digits
_, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY_INV)

# Find digit contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

digit_boxes = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w > 10 and h > 10:  # Filter out small noise
        digit_boxes.append((x, y, w, h))

# Sort boxes left to right
digit_boxes = sorted(digit_boxes, key=lambda box: box[0])

predicted_number = ""

for x, y, w, h in digit_boxes:
    roi = thresh[y:y+h, x:x+w]
    roi_resized = cv2.resize(roi, (28, 28))
    roi_normalized = roi_resized / 255.0
    roi_reshaped = roi_normalized.reshape(1, 28, 28, 1)

    pred = model.predict(roi_reshaped)
    digit = np.argmax(pred)
    predicted_number += str(digit)

print("✅ Predicted Number:", predicted_number)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
✅ Predicted Number: 2
